# Test Query
Purpose is to perform a query against the index created.
This is only meant to test the retrieval portion of the RAG application.

## Required for this step
- Azure AI Search Service



In [ ]:
# # Import required libraries  
import os  
import json
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import (
    QueryType,
    VectorizedQuery,
)
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt 


In [ ]:
#Load the configuration details for the Cognitive Search Service and Azure OpenAI Instance
#Credentials should be secured using a more secure method such as Azure KeyVault
from dotenv import load_dotenv
load_dotenv()

# Load configuration from environment variables
config = { 
    "search_service_name": os.getenv("SEARCH_SERVICE_NAME"),
    "search_admin_key": os.getenv("SEARCH_ADMIN_KEY"),
    "search_index_name": os.getenv("SEARCH_INDEX_NAME"),
    "search_index_schema_file": os.getenv("SEARCH_INDEX_SCHEMA_FILE"),
    "search_api_version": os.getenv("SEARCH_API_VERSION", "2023-07-01"),
    "data_root_dir": os.getenv("DATA_ROOT_DIR", "./data"),
    "openai_api_base": os.getenv("OPENAI_API_BASE"),
    "openai_api_key": os.getenv("OPENAI_API_KEY"),
    "openai_api_version": os.getenv("OPENAI_API_VERSION", "2023-05-15"),
    "openai_embedding_model": os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
}


# Azure AI Search Config
search_service_name = config["search_service_name"]
search_service_url = "https://{}.search.windows.net/".format(search_service_name)
search_admin_key = config["search_admin_key"]
index_name = config["search_index_name"]
index_schema_file = config["search_index_schema_file"]
search_api_version = config["search_api_version"]
search_headers = {  
    'Content-Type': 'application/json',  
    'api-key': search_admin_key  
}  

index_client = SearchIndexClient(
        endpoint=search_service_url, credential=AzureKeyCredential(search_admin_key))
search_client = SearchClient(endpoint=search_service_url, index_name=index_name, credential=AzureKeyCredential(search_admin_key))

print ('Search Service Name:', search_service_name)
print ('Index Name:', index_name)

#Azure OpenAI
api_base = config["openai_api_base"]
api_key = config["openai_api_key"]
openai_api_version = config["openai_api_version"]
embedding_model = config["openai_embedding_model"]

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    api_version=openai_api_version,
    azure_endpoint=api_base,
    api_key=api_key
)


In [ ]:
# Function to generate embeddings for title and content fields, also used for query embeddings
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model=embedding_model
    )
    return json.loads(response.model_dump_json())["data"][0]['embedding']

# generate_embeddings('test')

In [ ]:
# Pure Hubrid Search
query = "how do you value a company??"  
  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  
    print("==============================\n")

In [ ]:
# Pure Hybrd+Semantic Search
query = "how many shares were repurchased in the second quarter of 2022 by Microsoft?"  
  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    query_type=QueryType.SEMANTIC, semantic_configuration_name='vector-semantic-configuration',
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  
    print("==============================\n")